# Test for NeRmo2
This notebook contains necessary tests to help users getting familiar with NeRmo faster. 

The model is written in xml and it will be handled here with MuJoCo and MJX. 

The following things will be contained: 
- Joints
- Actuators
- Sensors
- Other special structures

## Revisions for the model
These revisions are necessary for the model to be implemented on MJX, since MJX does not handle some specific collisions, e.g., cylinders and boxes. 

- Remove `<flag override="enable"/>` in `basic_scene.xml`
- Set `m1_tail` geom from `cylinder` to `capsule` in `tail.xml`
- Four foot sensors `cylinder` -> `capsule` in `leg_xx_assets/xx_asset_exp.xml` (`xx=fl,fr,rl,rr`)
  - Format of the sensors: `foot_s_xx` (with `xx` as the placeholder)
- Cancel the contact by setting `contype` to 0 by adjusting `spec` of the model for:
  - `m_ss`: the servo for the spine
  - `servo_check_xx`
  - `thigh_up_link_1_xx`
  - `thigh_up_link_2_xx`
    - (`xx=fl,fr,rl,rr`)
- Enable full tail functions
  - `tail_assets/tail.xml` -> `tail_assets/tail_new.xml`
  - `tail_assets/tail_tendon.xml` -> `tail_assets/tail_tendon_new.xml`

In [ ]:
# import necessary modules

## modules for mujoco
import mujoco as mj
from mujoco import mjx

## computational modules
import jax
from jax import numpy as jp
import numpy as np

## modules for visualization
import matplotlib.pyplot as plt
import mediapy as media
import time
import imageio.v3 as iio

# basic constants
MODEL_PATH = "./dynamic_4l.xml"

Current knowledge: 
* Four foot sensors can be safely replaced by **capsule**s. By visualizing these sensors, they just coincide with the extremities of the four feet. 
  * `foot_s_fl, foot_s_fr, foot_s_rl, foot_s_rr`
  * done.
* The cylinder in `tail.xml` can be safely replaced by a **capsule**. 
  * done.
* The cylinder for `m_ss` can have its contype to be 0
* `servo_check_fl` as a cylinder will raise no collision. Thus their contype can all be set to 0. 

In [ ]:
# load mjspec from file and adjust spec
# dir(mj.MjSpec) # use mj.MjSpec.from_file
spec = mj.MjSpec.from_file(MODEL_PATH)
# try to get the geom m_ss, which is located at servo spine
m_ss = spec.geom("m_ss")
# m_ss.rgba[3] = 0
# try to set contype of m_ss to 0: success; does not influence the control
m_ss.contype = 0

suffix_list = ["fl", "fr", "rl", "rr"]
for suf in suffix_list:
    serfo_check = spec.geom(f"servo_check_{suf}")
    serfo_check.contype = 0
    thigh_up_link_1 = spec.geom(f"thigh_up_link_1_{suf}")
    thigh_up_link_1.contype = 0
    thigh_up_link_2 = spec.geom(f"thigh_up_link_2_{suf}")
    thigh_up_link_2.contype = 0

Try to use the tail... success! 
- `tail.xml` -> `tail_new.xml`
- `tail_tendon.xml` -> `tail_tendon_new.xml`

In [ ]:
# compile the model
mj_model = spec.compile()
mj_data = mj.MjData(mj_model)
renderer = mj.Renderer(mj_model)

The following blocks are for rendering purposes:

In [ ]:
# finally render the model
import mujoco.viewer
# mj.viewer.launch(mj_model, mj_data)

# launch a viewer that pause at the beginning
# with mj.viewer.launch_passive(mj_model, mj_data) as viewer:
#     time.sleep(10)
viewer = mj.viewer.launch_passive(mj_model, mj_data) # in this way one can see the default configuration

In [ ]:
# a separate button for closure
viewer.close()

In [ ]:
# a running launcher
mj.viewer.launch(mj_model, mj_data)

## Joints
This sections lists all important joints of the model. (Better use a figure to demonstrate! )

List the joints for sensors and actuators first. 

List the following attributes of joints for quick reference: 
- `ref`
- `damping`
- `armature`

## Sensors

This section lists all sensors of the model. All information is concluded from the xml files. 

Since the types of sensors in MuJoCo are rich, one can feel free to add new sensors for any desired information. 

List the following types of sensors for quick reference. 
- `jointpos`: measures the angle of the joint (returns a scalar) (copied from `mjData.sensordata`)
- `touch`: measures the contact force (returns a scalar)
- `framepos`: returns global 3D position of the site
- `framequat`: returns global orientation of the site
- `framelinvel`: returns global 3D linear velocity of the site
  - `velocimeter`: returns 3D linear velocity *in local coordinates* (not used here)
- `accelerometer`: returns 3D acceleration of the site *in local coordinates*
- `gyro`: returns angular velocity of the site *in local coordinates*

Sensors: 
- Sensors for servos on four legs (joint angle)
  - `thigh_joint_fl`
  - `leg_joint_fl`
  - `thigh_joint_fr`
  - `leg_joint_fr`
  - `thigh_joint_rl`
  - `leg_joint_rl`
  - `thigh_joint_rr`
  - `leg_joint_rr`
- Sensors for other servos (joint angle)
  - `m1_tail`
  - `neck`
  - `head`
  - `spine`
- Sensors for the angle of upper knees (joint angle)
  - `knee_fl`
  - `knee_fr`
  - `knee_rl`
  - `knee_rr`
- Sensors for feet (contact force)
  - `fl_t1`
  - `fr_t1`
  - `rl_t1`
  - `rr_t1`
- Sensors for body status
  - `com_pos`: global position of body
  - `com_quat`: quaternion of global orientation of body 
  - `com_vel`: global velocity of body
  - `imu_acc`: local acceleration (actually global, since it is on the outmost body)
  - `imu_gyro`: angular velocity of the site
- Sensors for feet (position of extremities)
  - `fl_foot_pos`
  - `fr_foot_pos`
  - `rl_foot_pos`
  - `rr_foot_pos`

## Actuators

This section lists all actuators of the model. 

List the following types of actuators for quick reference.
* `position`
* `velocity`
  * The two actuators can together make a PD controller.

Actuators
* Actuators for the four legs (position; velocity)
  * `leg_joint_fl`
  * `leg_joint_fl_vel`
  * `thigh_joint_fl`
  * `thigh_joint_fl_vel`
  * `leg_joint_fr`
  * `leg_joint_fr_vel`
  * `thigh_joint_fr`
  * `thigh_joint_fr_vel`
  * `leg_joint_rl`
  * `leg_joint_rl_vel`
  * `thigh_joint_rl`
  * `thigh_joint_rl_vel`
  * `leg_joint_rr`
  * `leg_joint_rr_vel`
  * `thigh_joint_rr`
  * `thigh_joint_rr_vel`
* Actuators for other joints (position)
  * `m1_tail`
  * `neck`
  * `head`
  * `spine`

## Test on MJX
Carry out some basic tests on MJX; by the way, list some of the common interfaces of MJX. 

In [ ]:
# place them on GPU device using MJX
mjx_model = mjx.put_model(mj_model)
mjx_data = mjx.put_data(mj_model, mj_data)
# print(mj_data.qpos, type(mj_data.qpos))
print(mjx_data.qpos, type(mjx_data.qpos), mjx_data.qpos.devices())

Run some simulations. 

_Problem_: `RuntimeWarning: overflow encountered in cast
  return literals.TypedNdArray(np.asarray(x, dtype), weak_type=False)`
- It seems that the problem is not that severe. 
- The running below is very slow on CPU. Write corresponding tests on the machine with GPU.

In [ ]:
# enable joint visualization option:
scene_option = mj.MjvOption()
scene_option.flags[mj.mjtVisFlag.mjVIS_JOINT] = True

duration = 3.8  # (seconds)
framerate = 60  # (Hz)

jit_step = jax.jit(mjx.step) # get a new function with fast speed

frames = []
# initialization step for mjx_data: reset mj_data, the cal mjx.put_data
mj.mj_resetData(mj_model, mj_data)
mjx_data = mjx.put_data(mj_model, mj_data)


In [ ]:
# small test about jit_step
mjx_data = jit_step(mjx_model, mjx_data)

In [ ]:

while mjx_data.time < duration:
    mjx_data = jit_step(mjx_model, mjx_data) # similar to mj.mj_step; 
    # step above very slow on CPU ...
    # the following is for rendering
    if len(frames) < mjx_data.time * framerate:
        mj_data = mjx.get_data(mj_model, mjx_data) # get mj_data for rendering
        renderer.update_scene(mj_data, scene_option=scene_option)
        pixels = renderer.render() # np array
        frames.append(pixels)

media.show_video(frames, fps=framerate)

# Other test
The following contents contains tests irrelevant to the model. 

## Accelerometer and framelinacc
These two accelerators measures linear acceleration, yet globally and locally, respectively. 

1. Globall measurement is obvious: in global frame. 
2. Local measurement means measure in local frame. 
3. "Including gravity" actually means the gravity is not taken into account, since a geom floating in the air steadily has acceleration along z axis, which just equals gravity acceleration in number (positive). 
   * It means that an object must has constant positive acceleration along z axis if its z coordinate stays unchanged.
   * Real acceleration needs subtracting z acceleration by gravity.

In [ ]:
import mujoco.viewer

# reset data 
xml_pure_rotation = """
<mujoco>
  <worldbody>
    <!-- Platform rotating in horizontal plane -->
    <light pos="0 0 30" dir="0 0 -1" directional="true"/>
    <geom type="plane" size="0 0 0.03"  rgba="1 1 1 1"/>

    <body name="obj" pos="0 0 0.3">
        <freejoint/>
        <geom type="box" size="0.1 0.1 0.1"/>
    </body>

    <body name="platform" pos="0 0 1">
      <joint name="rotator" type="hinge" axis="0 0 1"/>
      <geom type="cylinder" size="1.0 0.05" rgba="0.3 0.3 0.3 0.7"/>
      
      <!-- Sensor at edge of platform -->
      <site name="edge_sensor" pos="1.0 0 0.05" size="0.03"/>
      
      <!-- Object sitting on platform -->
      <body name="object_on_platform" pos="0.8 0 0.1">
        <geom type="box" size="0.1 0.1 0.1" rgba="0 0 1 0.7"/>
      </body>
    </body>
  </worldbody>
  
  <actuator>
    <motor name="spin_motor" joint="rotator" gear="100"/>
  </actuator>
  
  <sensor>
    <!-- 1. Accelerometer on rotating platform edge -->
    <accelerometer name="rotating_accel" site="edge_sensor"/>
    
    <!-- 2. Frame acceleration of the object ON the platform -->
    <framelinacc name="object_world_accel" objtype="body" objname="obj"/>
    
    <!-- 3. Frame acceleration of platform itself -->
    <framelinacc name="platform_world_accel" objtype="body" objname="platform"/>
  </sensor>
</mujoco>
"""

model = mujoco.MjModel.from_xml_string(xml_pure_rotation)
data = mujoco.MjData(model)
mujoco.viewer.launch(model, data)